In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [16]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [17]:
soup = BeautifulSoup(source, 'xml')

In [18]:
table=soup.find('table')

In [19]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [20]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [21]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
df=df[df['Borough']!='Not assigned']

In [26]:
df.loc[df['Neighborhood']=='Not assigned', 'Neighbourhood']=df['Borough']
df.head()

,Postalcode,Borough,Neighborhood,Neighbourhood
2,M3A,North York,Parkwoods,NaN
3,M4A,North York,Victoria Village,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN
5,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN


In [32]:
df1=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df1=df1.reset_index(drop=False)
df1.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [33]:
df2 = pd.merge(df, df1, on='Postalcode')
df2.drop(['Neighborhood'],axis=1,inplace=True)
df2.drop_duplicates(inplace=True)
df2.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df2.head()

,Postalcode,Borough,Neighbourhood,Neighborhood
0,M3A,North York,NaN,Parkwoods
1,M4A,North York,NaN,Victoria Village
2,M5A,Downtown Toronto,NaN,"Regent Park, Harbourfront"
3,M6A,North York,NaN,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,NaN,"Queen's Park, Ontario Provincial Government"


In [34]:
df2.shape

(103, 4)